In this section of the code, we train four discrete Machine and Deep learning models with the express purpose of forecasting traffic flow along every path encompassed within the time series dataset. Consequently, we conduct a meticulous analysis of the vehicular flux within the city of San Francisco for each pathway, under the guidance and enforcement of the SQP rules.

<b>The SPQ rules are fully described here:</b>https://dl.acm.org/doi/abs/10.1145/2666310.2666413

### Phase 4: Preprocess the time series dataset and add some weather data on it
In this step, the following commands are executed:
- Load time series dataset that created using SPQ function
- Preprocess the time series dataset
- Read weather data and preprocess them
- Aggregate traffic flow data with weather data
- Make visualizations of traffic flow per day and per 3-hour interval
- Extract features that are helpful to our research

In [1]:
# measure execution time
%load_ext autotime

# disable warnings
import warnings
warnings.filterwarnings('ignore')

# standard library imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime, timedelta

time: 1.66 s (started: 2023-07-25 19:02:39 +03:00)


#### Step 1: Load the time series traffic flow dataset that created using SPQ function
In this step we are doing the following operations:
- Read the data
- Change the name of columns

In [2]:
# read dataset
time_series = pd.read_csv('C:/Users/SK/Desktop/Πτυχιακή/Files/time_series.txt')

# this list contains the column names
columns = ["Taxi ID","Traj ID","Path","Length"]

# generate the columns of the dataset
i =4
while(True):
    if i == 4:
        columns.append(pd.to_datetime('2008-05-18 00:00:00'))
    else:
        columns.append(columns[i-1] + timedelta(seconds=1800))
    
    if (columns[i]>=pd.to_datetime('2008-05-24 23:59:59.000130')):
        break
    
    i+=1

# delete the last timestamp
columns.pop()

# assign new column names to our dataframe
time_series.columns = columns

time: 141 ms (started: 2023-07-25 19:02:41 +03:00)


#### Step 2: Preprocess the time series dataset
In this step we are doing the following operations:
- Reshape it to long format using melt function
- Encode categorical values
- Preprocess the data types of each column

In [3]:
# reshape the data from wide format to long format
id_cols = ['Taxi ID','Traj ID', 'Path', 'Length']
time_cols = time_series.iloc[:,2:].columns
time_series = time_series.melt(id_vars=id_cols, value_vars=time_cols, var_name='Time Column', value_name='Traffic Flow')
time_series['Time Column'] = pd.to_datetime(time_series['Time Column'])

# sort rows by Path and Time
time_series.sort_values(by=['Path','Time Column'],inplace=True)

time: 547 ms (started: 2023-07-25 19:02:42 +03:00)


In [ ]:
# convert path column from categorical to numerical
from sklearn.preprocessing import LabelEncoder

# create an instance of LabelEncoder
label_encoder = LabelEncoder()

# fit label encoder on the 'Path' column
time_series['Path'] = label_encoder.fit_transform(time_series['Path'])

# sort the data based in Path column
time_series = time_series.sort_values(by=['Path','Time Column'])

# print data
time_series.head()

#### Step 3: Read and preprocess the weather data
In the weather dataset of San Francisco area, there are many columns that are useless for our research. We just drop them and preprocess the dataset. 

In this step we are doing the following operations:
- Read the weather data
- Drop uneccessary informatio n
- Encode categorical attributes
- Fill Nan values

In [4]:
# read weather data
weather_data = pd.read_csv('C:/Users/SK/desktop/Πτυχιακή/weather_data.csv',sep=',')

# drop unecessary columns. 
# dropped columns can either have NaN values, or not be associated with traffic flow.
# columns kept: temperature, humidity, windspeed, sealevelpressure, visibility
weather_data = weather_data.drop(['name','feelslike','dew','precip','precipprob','preciptype',
                                  'snow','snowdepth','windgust','winddir','cloudcover',
                                  'solarradiation','solarenergy','uvindex','severerisk','stations','icon'],axis=1)

time: 47 ms (started: 2023-07-25 19:02:47 +03:00)


There is one column that contains categorical vales. Use one-hot encoding to represent them as numerical vectors.

In [5]:
# view the values of "conditions" column
weather_data['conditions'].value_counts()

Partially cloudy    435
Clear               166
Overcast            142
Rain, Overcast        1
Name: conditions, dtype: int64

time: 15 ms (started: 2023-07-25 19:02:49 +03:00)


In [6]:
# change the only 'Rain, Overcast' value to 'Overcast'
weather_data.loc[weather_data['conditions'] == 'Rain, Overcast', 'conditions'] = 'Overcast'

time: 0 ns (started: 2023-07-25 19:02:50 +03:00)


In [7]:
# view again the values of "conditions" column
weather_data['conditions'].value_counts()

Partially cloudy    435
Clear               166
Overcast            143
Name: conditions, dtype: int64

time: 16 ms (started: 2023-07-25 19:02:50 +03:00)


In [8]:
# one-hot encode the  "conditions" column
conditions = pd.get_dummies(weather_data['conditions'], prefix='conditions')
weather_data.drop('conditions',axis=1,inplace=True)

# concatenate the one-hot encoded columns to the original DataFrame
weather_data = pd.concat([weather_data, conditions], axis=1)

time: 0 ns (started: 2023-07-25 19:02:51 +03:00)


Now, there are not categorical values.

In [9]:
# view for null values
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   datetime                     744 non-null    object 
 1   temp                         744 non-null    float64
 2   humidity                     744 non-null    float64
 3   windspeed                    744 non-null    float64
 4   sealevelpressure             744 non-null    float64
 5   visibility                   744 non-null    float64
 6   conditions_Clear             744 non-null    uint8  
 7   conditions_Overcast          744 non-null    uint8  
 8   conditions_Partially cloudy  744 non-null    uint8  
dtypes: float64(5), object(1), uint8(3)
memory usage: 37.2+ KB
time: 16 ms (started: 2023-07-25 19:02:51 +03:00)


There are not null values in the weather dataset too!

In [10]:
# covnert datetime column to appropriate format
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# drop rows that are not in within the interval [2008-05-18,2008-05-24]
start_date = pd.to_datetime('2008-05-18')
end_date = pd.to_datetime('2008-05-25')

weather_data = weather_data[(weather_data['datetime'] >= start_date) & (weather_data['datetime'] <= end_date)].reset_index(drop=True)

time: 15 ms (started: 2023-07-25 19:02:52 +03:00)


#### Step 4: Connect the two dataframes to one
Use time information to correctly connect the weather data and traffic flow data together.

In [11]:
# connect two dataframes, time-series traffic flow and weather data
time_series.set_index('Time Column', inplace=True)
weather_data.set_index('datetime', inplace=True)

# resample the 'weather_data' dataset to 30-minute intervals and forward-fill the missing values
weather_data = weather_data.resample('30T').ffill()

# merge the time_series and weather_data_resampled datasets based on their indexes
time_series_weather = time_series.merge(weather_data, left_index=True, right_index=True)

time: 141 ms (started: 2023-07-25 19:02:53 +03:00)


#### Step 5: Feature Creation From Time Information
In this step we define some features based on time series timestamps.
- Day, dayfweek, hour and minute features derive directly from the timestamp column of the dataset.
- Hour_sin and Hour_cos columns transform the hour values into a cyclic representation, where the values range from -1 to 1. This transformation is used in time series analysis to capture the periodic patterns or cyclical nature of time-related data.
- Use also the 3_hour_interval to describe, in which interval of the day data refer to.

In [12]:
time_series_weather = time_series_weather.reset_index()
time_series_weather.rename(columns={'index': 'Time Column'}, inplace=True)

time: 32 ms (started: 2023-07-25 19:02:58 +03:00)


In [13]:
### extract timestamp information to different columns ###

time_series_weather['hour'] = time_series_weather['Time Column'].dt.hour
time_series_weather['dayofweek'] = time_series_weather['Time Column'].dt.dayofweek
time_series_weather['day'] = time_series_weather['Time Column'].dt.day
time_series_weather['minute'] = time_series_weather['Time Column'].dt.minute

### circular Encoding for cyclic time features ###

time_series_weather['hour_sin'] = np.sin(2 * np.pi * time_series_weather['hour'] / 24)
time_series_weather['hour_cos'] = np.cos(2 * np.pi * time_series_weather['hour'] / 24)

time_series_weather['day_of_week_sin'] = np.sin(2 * np.pi * time_series_weather['dayofweek'] / 7)
time_series_weather['day_of_week_cos'] = np.cos(2 * np.pi * time_series_weather['dayofweek'] / 7)

time_series_weather['day_sin'] = np.sin(2 * np.pi * time_series_weather['day'] / 31)
time_series_weather['day_cos'] = np.cos(2 * np.pi * time_series_weather['day'] / 31)

time_series_weather['minute_sin'] = np.sin(2 * np.pi * time_series_weather['minute'] / 60)
time_series_weather['minute_cos'] = np.cos(2 * np.pi * time_series_weather['minute'] / 60)

time: 266 ms (started: 2023-07-25 19:02:59 +03:00)


In [14]:
# define a custom function to determine the three-hour interval, based in timestamp hour information
def get_3hour_interval(hour):
    if hour in [0, 1, 2]:
        return 1
    elif hour in [3, 4, 5]:
        return 2
    elif hour in [6, 7, 8]:
        return 3
    elif hour in [9, 10, 11]:
        return 4
    elif hour in [12, 13, 14]:
        return 5
    elif hour in [15, 16, 17]:
        return 6
    elif hour in [18, 19, 20]:
        return 7
    elif hour in [21, 22, 23]:
        return 8
    else:
        return None   

time: 0 ns (started: 2023-07-25 19:03:00 +03:00)


In [15]:
# apply the custom function on the data to create the '3hour_interval' column
time_series_weather['3hour_interval'] = time_series_weather['hour'].apply(get_3hour_interval)

time: 312 ms (started: 2023-07-25 19:03:00 +03:00)


In [16]:
# print dataset
time_series_weather

,Time Column,Taxi ID,Traj ID,Path,Length,Traffic Flow,temp,humidity,windspeed,sealevelpressure,...,minute,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos,minute_sin,minute_cos,3hour_interval
0,2008-05-18 00:00:00,97,297,"[100400941, 100400941, 100400941, 100400941, 1...",8,4,12.4,87.29,16.1,1017.7,...,0,0.000000,1.000000,-0.781831,0.623490,-0.485302,-0.874347,0.000000e+00,1.0,1
1,2008-05-18 00:00:00,2,83,"[100400941, 100400941, 144346316]",3,4,12.4,87.29,16.1,1017.7,...,0,0.000000,1.000000,-0.781831,0.623490,-0.485302,-0.874347,0.000000e+00,1.0,1
2,2008-05-18 00:00:00,247,370,"[100400946, 91181759, 91181759, 91181759, 9118...",10,7,12.4,87.29,16.1,1017.7,...,0,0.000000,1.000000,-0.781831,0.623490,-0.485302,-0.874347,0.000000e+00,1.0,1
3,2008-05-18 00:00:00,327,78,"[1011710196, 997142479]",2,5,12.4,87.29,16.1,1017.7,...,0,0.000000,1.000000,-0.781831,0.623490,-0.485302,-0.874347,0.000000e+00,1.0,1
4,2008-05-18 00:00:00,363,56,"[1020512097, 517260065, 517260065, 416877246, ...",11,0,12.4,87.29,16.1,1017.7,...,0,0.000000,1.000000,-0.781831,0.623490,-0.485302,-0.874347,0.000000e+00,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335995,2008-05-24 23:30:00,403,113,"[985061724, 985061724, 985061724, 985061719, 9...",6,2,12.4,72.31,10.5,1015.0,...,30,-0.258819,0.965926,-0.974928,-0.222521,-0.988468,0.151428,5.665539e-16,-1.0,8
335996,2008-05-24 23:30:00,477,44,"[99231463, 99231463, 680402688, 680402688, 120...",9,0,12.4,72.31,10.5,1015.0,...,30,-0.258819,0.965926,-0.974928,-0.222521,-0.988468,0.151428,5.665539e-16,-1.0,8
335997,2008-05-24 23:30:00,135,125,"[996695182, 996695182]",2,3,12.4,72.31,10.5,1015.0,...,30,-0.258819,0.965926,-0.974928,-0.222521,-0.988468,0.151428,5.665539e-16,-1.0,8
335998,2008-05-24 23:30:00,192,164,"[997349781, 99278921, 99278921, 923712624, 923...",12,0,12.4,72.31,10.5,1015.0,...,30,-0.258819,0.965926,-0.974928,-0.222521,-0.988468,0.151428,5.665539e-16,-1.0,8


time: 219 ms (started: 2023-07-25 19:03:01 +03:00)


#### Step 6: Make Visualizations

In this step, we are doing the following operations:
- View total traffic flow (sum of traffic flow of every path in the dataset) per day. In this way we understand patterns that traffic flow has.

- View total traffic flow per day and 3-hour interval of the same day, to understand in which time of each day Traffic Flow is on peak.

In [ ]:
import seaborn as sns

##### Visualise the Traffic Flow in every path of the dataset per day
Use the sum of traffic flow of every path at each timestep to generate a total overview about the data

In [ ]:
# group by timestamp (index) and calculate the sum of the traffic flow
grouped_df = time_series_weather['Traffic Flow'].groupby(time_series_weather['Time Column']).sum()

# add results to dataframe
grouped_df = pd.DataFrame(grouped_df,index=grouped_df.index)

In [ ]:
# add additional time information in the dataset
grouped_df['hour'] = grouped_df.index.hour
grouped_df['3hour_interval'] = grouped_df['hour'].apply(get_3hour_interval)
grouped_df['dayofweek'] = grouped_df.index.dayofweek

In [ ]:
# create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# create a plot to view the results
plt.figure(figsize=(15,5))
sns.lineplot(grouped_df,x=grouped_df.index,y='Traffic Flow',hue='dayofweek',marker='o', palette=dark_palette,linewidth=2.5)
plt.xlabel('Time')
plt.ylabel('Sum of Traffic Flow')
plt.title('Sum of Traffic Flow in Every Path of the Dataset Over Time')
plt.show()

Based on graph above, we see that for each day, there is a seasonality pattern. In the noon hours Traffic Flow is lower than in the mornnings or in the evenings.

##### Visualize traffic flow per day and per 3hour time inteval

In [ ]:
# traffic flow on 2008-05-18
pl = grouped_df[(grouped_df.index >= '2008-05-18 00:00:00') & (grouped_df.index < '2008-05-19 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-18")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-19
pl = grouped_df[(grouped_df.index >= '2008-05-19 00:00:00') & (grouped_df.index < '2008-05-20 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-19")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-20
pl = grouped_df[(grouped_df.index >= '2008-05-20 00:00:00') & (grouped_df.index < '2008-05-21 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-20")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-21
pl = grouped_df[(grouped_df.index >= '2008-05-21 00:00:00') & (grouped_df.index < '2008-05-22 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-21")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-22
pl = grouped_df[(grouped_df.index >= '2008-05-22 00:00:00') & (grouped_df.index < '2008-05-23 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-22")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-23
pl = grouped_df[(grouped_df.index >= '2008-05-23 00:00:00') & (grouped_df.index < '2008-05-24 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-23")
plt.xlabel("Time information every 30 minutes")

In [ ]:
# traffic flow on 2008-05-24
pl = grouped_df[(grouped_df.index >= '2008-05-24 00:00:00') & (grouped_df.index < '2008-05-25 00:00:00')]

# Create a custom dark color palette
dark_palette = sns.color_palette('dark', n_colors=8)

# plot the data
plt.figure(figsize=(15,5))
sns.lineplot(pl,x='Time Column',y='Traffic Flow',hue='3hour_interval',marker='o', palette=dark_palette,linewidth=2.5)
plt.title("Traffic Flow on 2008-05-24")
plt.xlabel("Time information every 30 minutes")

##### Find highly correlated data in the dataset

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(time_series_weather.corr(), annot=True, cmap='viridis')

Based on the above heatmap, we see that there are some features that are highly correlated. For example,
temperature is correlated wiith humidity, windspeed, the hour of day, 3hour_interval and circular encoding of hour feature. Based on this heatmap above, we can choose the features that will be used in the final model.

### Phase 5: Build machine learning models for traffic flow forecasting
In this step, the following commands are executed:
- Train and Test split
- Find the best lookback timesteps for our XGBoost model
- Perform Grid Search CV to find the optimum parameters for the XGBoost model
- Train XGboost model (1)
- Train an LSTM model (2)
- Train a random forest model (3)
- Perform Grid Search CV to find the optimum parameters for the random forest model
- Train Encodel-Decoder model (4)
- Visualise the losses for each model
- Test on testing set each model
- Add in the same dataframe the scores of each model, in order to compare them
- Make visualizations on actual vs predicted traffic flow data

#### Step 1: Train - Test split
In this step, we split the data into two datasets train and test.
The time inteval of all observations in this dataset are within [18-05-2008, 24-05-2008].
- Train dataset contains all the data for each path till 2008-05-23 inclusive.
- The rest of the data (the most resent ones) are in the test dataset

In [ ]:
# sort the time series data
time_series_weather.sort_values(by=['Path','Time Column'], inplace=True)

In [ ]:
# define train data
train = time_series_weather.loc[time_series_weather['Time Column'] < '2008-05-24']

# define test data
test = time_series_weather.loc[time_series_weather['Time Column'] >= '2008-05-24']

# print the results
print("Train set shape: ",train.shape)
print("Test set shape: ",test.shape)

In [ ]:
# redefine train and test sets based on correlated features (drop highly correlated features)
train = train.drop(['Time Column','hour', 'hour_sin','day_of_week_cos', 'dayofweek', 'day', 'minute'],axis=1)
test = test.drop(['Time Column','hour', 'hour_sin','day_of_week_cos', 'dayofweek', 'day', 'minute'],axis=1)

#### Step 2: Find the best lookback timesteps for our XGBoost model
In this step, we define the optimum lookback timestep number for our sliding window. This optimum number will be used by the model for forecasting. Also, since XGBoost is the best model based in RMSE score, we perform this operation only for this model.

##### Step 2a: Create empty dataframe to store the scores per lookback timesteps

In [ ]:
# find best lookback timesteps to perform analysis. Save those scores in a separate dataframe
lookback_scores = pd.DataFrame(columns=['#Timesteps','XGBoost Score'])
lookback_scores['#Timesteps'] = 0
lookback_scores['XGBoost Score'] = 0

##### Step 2b: Create function that converts the data into a supervised problem
Consider for each path n_in timesteps of traffic flow in the past, in order to predict the n_out traffic flow timesteps in the future.

In [ ]:
from pandas import DataFrame
from pandas import concat

# create functio for generating (X, y) pairs
def series_to_supervised(data, n_in=1, n_out=1):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('Traffic Flow %d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('Traffic Flow %d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('Traffic Flow %d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    return agg

In [ ]:
"""
Convert the training data into a supervised dataset. Create X,y pairs of the traffic column information.
Then, concatenate the rest of the culumns. 
"""
def generate_X_y_pairs(data,n_in=1,n_out=1):
    new_data = pd.DataFrame()
    for item in data['Path'].unique():
        
        # call function only for traffic flow columns
        supervised_traffic_flow = series_to_supervised(pd.DataFrame(data[data['Path'] == item]['Traffic Flow']), n_in, n_out)
        
        # the rest of the columns are concatenated as they were
        supervised_traffic_flow = pd.concat([supervised_traffic_flow, time_series_weather[time_series_weather['Path'] == item]], axis=1)
        
        new_data = pd.concat([new_data,supervised_traffic_flow])

    new_data.dropna(inplace=True)
    return new_data

##### Step 2c: Create a loop. In each loop, we are doing the following:
- 0. Convert train and test sets into supervised problem with lookback timesteps.
- 1. Define features and labels.
- 2. Train the XGboost model with defult parameters
- 3. Predict on the test set
- 4. Save RMSE in a dataframe

In [ ]:
# import libraries
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
for lookback in [2,3,4,5,6]: # different lookback sizes
    
    # convert train and test data to supervised
    train_s = generate_X_y_pairs(train,lookback)
    test_s = generate_X_y_pairs(test,lookback)
    
    print("Lookback = "+str(lookback)+" - To supervised: Completed!")

    # add rolling mean information
    train_s['rolling_mean'] = 0
    test_s['rolling_mean'] = 0
    
    train_s['rolling_mean'] = train_s.iloc[:,0:lookback].mean(axis=1)
    test_s['rolling_mean'] = test_s.iloc[:,0:lookback].mean(axis=1)
    
    # define features
    features_s = train_s.drop(['Time Column','Traffic Flow','Traffic Flow 1(t)'],axis=1).columns
    
    # define label
    labels_s = ['Traffic Flow 1(t)']
    
    # create X_train and y_train datasets
    X_train_s = train_s[features_s]
    y_train_s = train_s[labels_s]
    
    # create X_test and y_test datasets
    X_test_s = test_s[features_s]
    y_test_s = test_s[labels_s]
    
    print("Lookback = "+str(lookback)+" - Train and Test sets: Completed!")

    # define XGBoost regression model
    reg = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=1000,early_stopping_rounds = 5)
    
    # train the XGBoost model with feature weights, use validation set on test data)
    reg.fit(X_train_s, y_train_s, eval_set=[(X_train_s, y_train_s), (X_test_s,y_test_s)],verbose=False)
    
    print("Lookback = "+str(lookback)+" - Trainning: Completed!")
    
    # make predictions
    y_pred_s = reg.predict(X_test_s)
    
    # calculate RMSE score
    rmse = mean_squared_error(y_test_s, y_pred_s, squared=False)
   
    # create a new row as a dictionary
    new_row = {'#Timesteps': lookback, 'XGBoost Score': rmse}

    # append the new row to the DataFrame
    lookback_scores = lookback_scores.append(new_row, ignore_index=True)
    
    print("Lookback = "+str(lookback)+" - Saved Results: Completed!")

#####  Step 2d: Print RMSE scores over lookback timesteps into a graph

In [ ]:
# plot the data
plt.plot(lookback_scores['#Timesteps'], lookback_scores['XGBoost Score'], linestyle='-', marker='o', color='b')

# set plot title and axis labels
plt.title('Lookback timesteps over RMSE score')
plt.xlabel('Lookback Timesteps of column Traffic Flow')
plt.ylabel('RMSE score after XGBoost testing')

# add gridlines
plt.grid(True)

# show the plot
plt.show()

In [ ]:
# find the timesteps corresponding to the minimum RMSE score
optimum_lookback = int(lookback_scores['#Timesteps'].loc[lookback_scores['XGBoost Score'].idxmin()])

#### Step 3: Perform Grid Search CV to find the optimum parameters for the XGBoost model
This step is essential to tune correctly the XGboost model.

##### Step 3a: Split train and test into supervised problem, based on optimum lookback timesteps that found above.

In [ ]:
# convert train and test data to supervised problem based in the optimum lookback step we defined above.
train = generate_X_y_pairs(train,optimum_lookback)
test = generate_X_y_pairs(test,optimum_lookback)

##### Step 3b: Extract features about trend of time series
In this step we define the rolling mean and rolling variance of a window. Since we are trying to forecast the last value of each window, only the first optimum_lookback values in the window are used to compute rolling features. The last value (that we are trying to forecast) is not included.

In [ ]:
# add rolling mean information
train['rolling_mean'] = 0
test['rolling_mean'] = 0

train['rolling_var'] = 0
test['rolling_var'] = 0

# calculate rolling mean of window
train['rolling_mean'] = train.iloc[:,0:optimum_lookback].mean(axis=1)
test['rolling_mean'] = test.iloc[:,0:optimum_lookback].mean(axis=1)

# calculate rolling variance of window
train['rolling_var'] = train.iloc[:, 0:optimum_lookback].var(axis=1)
test['rolling_var'] = test.iloc[:, 0:optimum_lookback].var(axis=1)

#####  Step 3c: Define labels and features
- Features will help the XGboost algorithm to predict the output value, the label. We will use all the traffic flow information at previous timesteps (t-n,t-n+1...,t-2,t-1), as well as the rest of the features (time and trend information) as features.
- Label will be out target value, the Traffic Flow at timestep t.

In [ ]:
# define features
features = train.drop(['Time Column','Traffic Flow 1(t)','Traffic Flow','hour', 'hour_sin','day_of_week_cos', 'dayofweek', 'day', 'minute'],axis=1).columns

# define labels
labels = ['Traffic Flow 1(t)']

# create X_train and y_train data sets
X_train = train[features]
y_train = train[labels]

# create X_test and y_test data sets
X_test = test[features]
y_test = test[labels]

In [ ]:
# show shape information about X and y training and testing sets
print("Shape of X_train is: ",X_train.shape)
print("Shape of y_train is: ",y_train.shape)
print("Shape of X_test is: ",X_test.shape)
print("Shape of y_test is: ",y_test.shape)

##### Step 3d: Apply GridSearchCV
Use a 5-fold K-fold cross-validation was used for learning the optimum hyperparameters for the XGBoost algorithm.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# tune hyperparameters using XGboost
xgb_params = {
    "n_estimators": [500],
    "learning_rate": [0.1],
    "max_depth": [3,9],
    "objective":['reg:squarederror'],
    "gamma": [0,1,2],
    "alpha": [0,0.1,1]
}

# define GridSearchCV object
xgb_grid = GridSearchCV(xgb.XGBRegressor(), 
                        xgb_params,cv=3,
                        verbose=1,
                        scoring= "neg_mean_squared_error")

# train on train data using 5 fold CV
xgb_grid.fit(X_train,y_train,
             early_stopping_rounds=5,
             eval_set=[(X_train, y_train), (X_test,y_test)],
             eval_metric='rmse')

# print the best hyperparameters and corresponding score
print("Best Hyperparameters: ", xgb_grid.best_params_)
print("Best Score: ", xgb_grid.best_score_)

#### Step 4: Create dataframe to save actual values and model predictions

In [ ]:
total_predictions = pd.DataFrame(index=pd.to_datetime(test['Time Column']))
total_predictions['Actual'] = y_test['Traffic Flow 1(t)'].values
total_predictions['Path'] = X_test['Path'].values

#### Step 5: Train an XGboost model
We will fit all the data in this model using the optimum parameters we found above. The optimum parameters for this dataset are: 
- n_estimators = 500
- learning_rate = 0.1
- max_depth = 9
- alpha = 0.1
- gamma = 1

Create XGBoost model and train it

In [ ]:
# define XGBoost regressor, use also gamma and alpha values for regularization
reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    early_stopping_rounds=5,
    max_depth=9 ,
    learning_rate=0.1,
    gamma=1,
    alpha=0.1    
)

# train the XGBoost model with feature weights, use validation set on test data)
reg.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)], 
        verbose=True)

Feature Importance of XGBoost model
Show how our model used the feautres, in order to perform splits. 

In [ ]:
# Show how much each of the features was used in our regression analysis
plt.figure(figsize=(15,5))
fi = pd.DataFrame(data=reg.feature_importances_,index=reg.feature_names_in_,columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.show()

Plot train and validation errors of XGBoost model in the same graph

In [ ]:
# acquire the values of train and validation errors
results = reg.evals_result()
train_errors = results['validation_0']['rmse']
validation_errors = results['validation_1']['rmse']

# plot the validation and training errors
plt.figure(figsize=(10, 6))
plt.plot(train_errors, label='Train Error (Train Set)')
plt.plot(validation_errors, label='Validation Error (Test Set)')
plt.xlabel('Number of Iterations')
plt.ylabel('RMSE score of XGBoost')
plt.title('XGBoost Model Training and Validation Errors')
plt.legend()
plt.show()

Make predictions on the test set

In [ ]:
y_pred_xgboost = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate the MAE of XGBoost model
mae_xgboost = mean_absolute_error(y_test, y_pred_xgboost)
print("Mae is: ",mae_xgboost)

# Calculate RMSE score of XGBoost model
rmse_xgboost = mean_squared_error(y_test, y_pred_xgboost, squared=False)
print("RMSE score:", rmse_xgboost)

Show example of how well the model learned the trainning data.


Use a random path to visualize the results

In [ ]:
# predict on the trainning set
y_pred_train = reg.predict(X_train)

# add information in the trainning set
train_predictions = pd.DataFrame(index=pd.to_datetime(train['Time Column']))
train_predictions.insert(0,'Train Actual',y_train['Traffic Flow 1(t)'].values)
train_predictions.insert(1,'Train Predicted',y_pred_train)
train_predictions.insert(2,'Path',X_train['Path'].values)

# plot an example of the train-predictions at path 0 of the datase
plt.figure(figsize=(15,5))
plt.plot(train_predictions[train_predictions['Path']==0]['Train Actual'],".-",label='Actual Values')
plt.plot(train_predictions[train_predictions['Path']==0]['Train Predicted'],".-",label='Predictions on train set using XGboost')
plt.xlabel('Time')
plt.ylabel('Traffic Flow in current path')
plt.title('Forecast on train set using XGBoost model')
plt.legend()
plt.show()

As we can see, the model has learned the trend and (almost) the seasonality of the data. 

Despite the non-linearity of Traffic Flow, XGBoost performs quite well!

Show an example of actual values vs predicted on test set

In [ ]:
test_predictions = pd.DataFrame(index=pd.to_datetime(test['Time Column']))
test_predictions.insert(0,'Test Actual',y_test['Traffic Flow 1(t)'].values)
test_predictions.insert(1,'Test Predicted',y_pred_xgboost)
test_predictions.insert(2,'Path',X_test['Path'].values)

# plot an example of the train-predictions
plt.figure(figsize=(15,5))
plt.plot(test_predictions[test_predictions['Path']==0]['Test Actual'],".-",label='Actual Values')
plt.plot(test_predictions[test_predictions['Path']==0]['Test Predicted'],".-",label='Predicted Values from XGBoost')
plt.xlabel('Time information')
plt.ylabel('Traffic Flow in current path')
plt.title('Actual Values vs. Predicted on random Path of test set (XGBoost model)')
plt.legend()
plt.show()

#### Step 6: Train LSTM for time series forecasting
This is the second model that we will train on the same dataset. We use the optimum lookback score that we computed above.

##### Step 6a: Create the train and test sets as before, then scale them
Use the same train and test sets as in the XGBoost model. In order to create more accurate predictions, we scale those two sets before giving them as input to the LSTM RNN model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# scale the data
scaler = MinMaxScaler()
X_tr = scaler.fit_transform(X_train)
X_te = scaler.fit_transform(X_test)

y_tr = scaler.fit_transform(y_train)
y_te = scaler.fit_transform(y_test)

In [ ]:
# create 3D train set for LSTM
trainX = []
trainY = []

for i in range(len(X_tr)):
    trainX.append(X_tr[i:i+1,:])
    
for i in range(len(y_tr)):
    trainY.append(y_tr[i:i+1,:])
    
trainX, trainY = np.array(trainX), np.array(trainY)

In [ ]:
# create 3D test set FOR LSTM
testX = []
testY = []

for i in range(len(X_te)):
    testX.append(X_te[i:i+1,:])
    
for i in range(len(y_te)):
    testY.append(y_te[i:i+1,:])
    
testX, testY = np.array(testX), np.array(testY)

In [ ]:
# print shapes of the created tests
trainX.shape, trainY.shape, testX.shape, testY.shape

##### Step 6b: Use the LSTM model for forecasting

Import libraries and define the LSTM structure

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import tensorflow as tf

In the LSTM model, we include dense layers to capture more complex non linear patterns and dropout layers to prevent overfitting.

In [ ]:
# Define LSTM model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(25, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(12, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Additional Dense layers
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='linear'))
model.add(Dense(trainY.shape[1]))  # Output layer


# Add early stopping mechanism
early_stopping = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)

Compile the model

In [ ]:
# compile LSTM model
model.compile(optimizer='adam', loss='mse')

Train the model

In [ ]:
# fit model on training data
history = model.fit(trainX, trainY, 
          epochs=50, 
          batch_size=len(trainX)//100,
          validation_data=(testX,testY), 
          verbose=1, 
          callbacks=[early_stopping])

In [ ]:
# evaluate model on test data
loss_lstm = model.evaluate(testX, testY)
print("Test Loss:", loss_lstm)

In [ ]:
### PLOT TRAIN VS. VALIDATION LOSS OF LSTM MODEL ###

# extract training and validation losses from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# plot the losses
plt.figure(figsize=(15,5))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss Value')
plt.title('Training Loss vs Validation Loss of LSTM model')
plt.legend()

# show the plot
plt.show()

Make predictions on test set

In [ ]:
# predict on test data
y_pred_lstm = model.predict(testX)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm.reshape(-1,1))

Evaluate the LSTM model

In [ ]:
# calculate the MAE score of LSTM model
mae_lstm = mean_absolute_error(scaler.inverse_transform(testY.reshape(-1,1)),y_pred_lstm)
print("Mae LSTM is: ",mae_lstm)

# Calculate the RMSE score of LSTM model
rmse_lstm = mean_squared_error(scaler.inverse_transform(testY.reshape(-1,1)), y_pred_lstm, squared=False)
print("RMSE LSTM score:", rmse_lstm)

Show example of how well the LSTM captured/learned the train data

In [ ]:
# predict on the trainning set
y_pred_train_LSTM = model.predict(trainX)

# add information in the trainning set
train_predictions = pd.DataFrame(index=pd.to_datetime(train['Time Column']))
train_predictions.insert(0,'Train Actual',y_train['Traffic Flow 1(t)'].values)
train_predictions.insert(1,'Train Predicted',scaler.inverse_transform(y_pred_train_LSTM.reshape(-1,1)))
train_predictions.insert(2,'Path',X_train['Path'].values)

# plot an example of the train-predictions
plt.figure(figsize=(15,5))
plt.plot(train_predictions[train_predictions['Path']==0][['Train Actual','Train Predicted']],".-")
plt.xlabel('Time')
plt.ylabel('Traffic Flow in current path')
plt.title('Forecast on train set using LSTM model')
plt.show()

#### Step 7: Create Random Forest model
This is the third model that we train on the same data, in order to forecast the traffic flow prediction. This algorithm is quite the same with the XGBoost, because both of them use decision trees to make forecasts.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

##### Step 7a: Find optimum parameters for random forest

In [ ]:
# define the parameter grid for the grid search
param_grid = {
    'max_depth': [5, 7],
    'min_samples_split': [5, 10],
    'n_estimators': [50]
}

# create the Random Forest model
rf_model = RandomForestRegressor(n_jobs=-1)

# create the GridSearchCV object
grid_search = GridSearchCV(rf_model, param_grid, cv=3, scoring='neg_mean_squared_error',verbose=1)

# fit the GridSearchCV object on the training data
grid_search.fit(X_train, y_train)

# get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# print the best parameters and the best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

##### Step 7b: Create and train best random forest model

In [ ]:
# get the best model after Grid Search
best_model = RandomForestRegressor(n_estimators=150,
                                  min_samples_split=10,
                                  max_depth=7,
                                  verbose=1,
                                  n_jobs=-1)

In [ ]:
best_model.fit(X_train,y_train)

##### Step 7c: Make predictions on train set

In [ ]:
# make predictions on the test set
y_pred_rf = best_model.predict(X_test)

In [ ]:
# predict on the trainning set using random forest
y_pred_train_rf = best_model.predict(X_train)

# add information in the trainning set
train_predictions = pd.DataFrame(index=pd.to_datetime(train['Time Column']))
train_predictions.insert(0,'Train Actual',y_train['Traffic Flow 1(t)'].values)
train_predictions.insert(1,'Train Predicted',y_pred_train_rf)
train_predictions.insert(2,'Path',X_train['Path'].values)

# plot an example of the train-predictions
plt.figure(figsize=(15,5))
plt.plot(train_predictions[train_predictions['Path']==0][['Train Actual','Train Predicted']],".-")
plt.xlabel('Time')
plt.ylabel('Traffic Flow in current path')
plt.title('Forecast on train set using random forest model')
plt.show()

##### Step 7d: Evalueate the random forest regressor

In [ ]:
# calculate the MAE
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print("MAE: ", mae_rf)

# calculate the RMSE
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
print("RMSE: ", rmse_rf)

#### Step 8: Train an Encoder-Decoder model
This is the fourth model that we are using, in order to forecast the traffic flow.

##### Step 8a: Initialize and compile the Encoder-Decoder model

In [ ]:
# add necessary imports
from keras import layers

In [ ]:
# convert train and test set to infinite datasets for faster training
BATCH_SIZE = 1
BUFFER_SIZE = 1000

train_univariate = tf.data.Dataset.from_tensor_slices((trainX, trainY))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
val_univariate = tf.data.Dataset.from_tensor_slices((testX, testY))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
### DEFINE THE MODEL ###

# Encoder part
enco_deco = Sequential()
enco_deco.add(LSTM(100, input_shape=(1, trainX.shape[2]), return_sequences=True))
enco_deco.add(Dropout(0.2))  # Add dropout to the first LSTM layer
enco_deco.add(LSTM(units=50, return_sequences=True))
enco_deco.add(Dropout(0.2))  # Add dropout to the second LSTM layer
enco_deco.add(LSTM(units=15))  

# Feature vector
enco_deco.add(layers.RepeatVector(1))  

# Decoder part
enco_deco.add(LSTM(units=100, return_sequences=True))
enco_deco.add(LSTM(units=50, return_sequences=True))
enco_deco.add(TimeDistributed(layers.Dense(units=1)))

Similart to the LSTM model, we include dense layers to capture more complex non linear patterns and dropout layers to prevent overfitting.

In [ ]:
enco_deco.compile(optimizer='adam', loss='mse')

##### Step 8b: Train the model and make predictions

In [ ]:
# train the enco-deco model
history = enco_deco.fit(train_univariate, 
                        epochs=50, 
                        steps_per_epoch=len(trainX)//100,
                        validation_data=val_univariate, 
                        validation_steps=len(testX)//100, 
                        verbose=1, 
                        callbacks=[early_stopping])

In [ ]:
# make predictions using Encoder-Decoder model
predicted_enco_deco = enco_deco.predict(testX)
predicted_enco_deco = scaler.inverse_transform(predicted_enco_deco.reshape(-1, 1))

##### Step 8c: Evaluate the model

In [ ]:
# Calculate MAE of Encoder - Decoder model
mae_enco_deco = mean_absolute_error(y_test, predicted_enco_deco)

# Calculate RMSE Encoder - Decoder model
rmse_enco_deco = mean_squared_error(y_test, predicted_enco_deco, squared=False)

print("MAE:", mae_enco_deco)
print("RMSE:", rmse_enco_deco)

Plot train loss vs. validation loss

In [ ]:
# Get the loss and validation loss values and pllot them in the same graph
loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss and validation loss
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss of Encoder-Decoder model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Forecast on train set using Encoder-Decoder model

In [ ]:
# predict on the trainning set
y_pred_train_enco_deco = enco_deco.predict(trainX)
y_pred_train_enco_deco = scaler.inverse_transform(y_pred_train_enco_deco.reshape(-1, 1))

# add information in the trainning set
train_predictions = pd.DataFrame(index=pd.to_datetime(train['Time Column']))
train_predictions.insert(0,'Train Actual',y_train['Traffic Flow 1(t)'].values)
train_predictions.insert(1,'Train Predicted',y_pred_train_enco_deco)
train_predictions.insert(2,'Path',X_train['Path'].values)

# plot an example of the train-predictions
plt.figure(figsize=(15,5))
plt.plot(train_predictions[train_predictions['Path']==0][['Train Actual','Train Predicted']],".-")
plt.xlabel('Time')
plt.ylabel('Traffic Flow in current path')
plt.title('Forecast on train set using Encoder-Decoder model')
plt.show()

#### Step 9: Add predictions of the four models to the same dataframe

In [ ]:
# store predictions of XGBoost model to a dataframe
total_predictions['Predicted XGBoost'] = y_pred_xgboost

In [ ]:
# store predictions of LSTM model to a dataframe
total_predictions['Predicted LSTM'] = scaler.inverse_transform(y_pred_lstm.reshape(-1,1))

In [ ]:
# add predictions of random forest
total_predictions['Predicted Random Forest'] = y_pred_rf

In [ ]:
# add predictions of encoder decoder model
total_predictions['Predicted Encoder Decoder Model'] = predicted_enco_deco

In [ ]:
# print dataframe with predictions
total_predictions

#### Step 10: Add in the same dataframe the scores of each model, in order to compare them

In [ ]:
# create dataframe with scores
scores = pd.DataFrame(columns=['Model','RMSE Score','MAE Score'])

# define rows for dataframe
new_row = {'Model': 'XGBoost', 'RMSE Score': rmse_xgboost, 'MAE Score': mae_xgboost}
new_row1 = {'Model': 'LSTM', 'RMSE Score': rmse_lstm, 'MAE Score': mae_lstm}
new_row2 = {'Model': 'Random Forest', 'RMSE Score': rmse_rf, 'MAE Score': mae_rf}
new_row3 = {'Model': 'Encoder Decoder', 'RMSE Score': rmse_enco_deco, 'MAE Score': mae_enco_deco}

# add rows to dataframe
scores = scores.append(new_row, ignore_index=True)
scores = scores.append(new_row1, ignore_index=True)
scores = scores.append(new_row2, ignore_index=True)
scores = scores.append(new_row3, ignore_index=True)

In [ ]:
# print dataframe with scores
scores

#### Step 11: Plot examples of actual vs Predicted for each of the models
In this part, we select a random path of the dataset and compare (for each model) the actual Traffic Flow values vs. the predicted ones

In [ ]:
# plot results from XGBoost
plt.figure(figsize=(15,5))
plt.plot(total_predictions[total_predictions['Path']==0]['Actual'],label='Actuals')
plt.plot(total_predictions[total_predictions['Path']==0]['Predicted XGBoost'],label='Predicted')
plt.xlabel("Time information")
plt.ylabel('Traffic Flow values')
plt.title('Actual vs Predicted traffic flow values of path 0 using XGBoost model')
plt.legend()
plt.plot()

In [ ]:
# plot results from LSTM
plt.figure(figsize=(15,5))
plt.plot(total_predictions[total_predictions['Path']==0]['Actual'],label='Actuals')
plt.plot(total_predictions[total_predictions['Path']==0]['Predicted LSTM'],label='Predicted')
plt.xlabel("Time information")
plt.ylabel('Traffic Flow values')
plt.title('Actual vs Predicted traffic flow values of path 0 using LSTM model')
plt.legend()
plt.plot()

In [ ]:
# plot results from Random Forest
plt.figure(figsize=(15,5))
plt.plot(total_predictions[total_predictions['Path']==0]['Actual'],label='Actuals')
plt.plot(total_predictions[total_predictions['Path']==0]['Predicted Random Forest'],label='Predicted')
plt.xlabel("Time information")
plt.ylabel('Traffic Flow values')
plt.title('Actual vs Predicted traffic flow values of path 0 using Random Forest model')
plt.legend()
plt.plot()

In [ ]:
# plot results from Encoder Decoder
plt.figure(figsize=(15,5))
plt.plot(total_predictions[total_predictions['Path']==0]['Actual'],label='Actuals')
plt.plot(total_predictions[total_predictions['Path']==0]['Predicted Encoder Decoder Model'],label='Predicted')
plt.xlabel("Time information")
plt.ylabel('Traffic Flow values')
plt.title('Actual vs Predicted traffic flow values of path 0 using Encoder-Decoder model model')
plt.legend()
plt.plot()

#### Step 12: Make visualizations on actual vs predicted traffic flow data
Use best model's predictions from total_predictions dataframe and plot the residuals between the actual traffic flow values and predicted. 

The graph to be created will contain the sum of traffic flow per timestamp, as done before in the visualization step.

In [ ]:
# groupd dataframe by timestamp
grouped = total_predictions.groupby(total_predictions.index).sum()

# add results to dataframe
grouped = pd.DataFrame(grouped,index=grouped.index)

# drop path column, as it is useless
grouped.drop('Path', axis=1, inplace=True)

##### Decide which model's predicted values will be used, based on the best RMSE score

In [2]:
# find the model with the best RMSE score (the lowest)
index = scores[['RMSE Score']].idxmin() # find the model with the minimun RMSE score (best model)

NameError: name 'scores' is not defined

In [ ]:
if (scores.iloc[index]['Model'][0] == 'XGBoost'):
    # create a plot to view the results
    plt.figure(figsize=(15,5))
    sns.lineplot(grouped,x=grouped.index,y='Actual',marker='o',linewidth=2.5,label='Actual')
    sns.lineplot(grouped,x=grouped.index,y='Predicted XGBoost',marker='o',linewidth=2.5, color='orange',label='XGboost')
    plt.xlabel('Time')
    plt.ylabel('Sum of Traffic Flow')
    plt.title('Sum of Traffic Flow in Every Path of the Dataset Over Time vs Predicted')
    plt.show()

elif (scores.iloc[index]['Model'][0] == 'LSTM'):
      # create a plot to view the results
      plt.figure(figsize=(15,5))
      sns.lineplot(grouped,x=grouped.index,y='Actual',marker='o',linewidth=2.5,label='Actual')
      sns.lineplot(grouped,x=grouped.index,y='Predicted XGBoost',marker='o',linewidth=2.5, color='orange',label='XGboost')
      plt.xlabel('Time')
      plt.ylabel('Sum of Traffic Flow')
      plt.title('Sum of Traffic Flow in Every Path of the Dataset Over Time vs Predicted')
      plt.show()

elif (scores.iloc[index]['Model'][0] == 'Random Forest'):
    # create a plot to view the results
      plt.figure(figsize=(15,5))
      sns.lineplot(grouped,x=grouped.index,y='Actual',marker='o',linewidth=2.5,label='Actual')
      sns.lineplot(grouped,x=grouped.index,y='Predicted XGBoost',marker='o',linewidth=2.5, color='orange',label='XGboost')
      plt.xlabel('Time')
      plt.ylabel('Sum of Traffic Flow')
      plt.title('Sum of Traffic Flow in Every Path of the Dataset Over Time vs Predicted')
      plt.show()

elif (scores.iloc[index]['Model'][0] == 'Encoder Decoder'):
    # create a plot to view the results
      plt.figure(figsize=(15,5))
      sns.lineplot(grouped,x=grouped.index,y='Actual',marker='o',linewidth=2.5,label='Actual')
      sns.lineplot(grouped,x=grouped.index,y='Predicted XGBoost',marker='o',linewidth=2.5, color='orange',label='XGboost')
      plt.xlabel('Time')
      plt.ylabel('Sum of Traffic Flow')
      plt.title('Sum of Traffic Flow in Every Path of the Dataset Over Time vs Predicted')
      plt.show()